In [5]:
import time

# Scrapping with Facebook 

In [4]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

# Dossier où les photos seront enregistrées
output_folder = r"C:\Users\Republic Of Computer\Documents\project_ai\facebook2"
os.makedirs(output_folder, exist_ok=True)

# Configuration de Selenium WebDriver
driver = webdriver.Chrome()  # Assurez-vous que chromedriver est dans votre PATH ou spécifiez le chemin complet

# Ouvrir la page Facebook
driver.get("https://web.facebook.com/marketplace/108144375883042/vehicles")

# Attendre que la page charge (ajustez le délai si nécessaire)
time.sleep(10)  # Ajustez selon les besoins pour laisser le temps à la page de se charger

# Récupérer le contenu de la page
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extraire les URLs des images
images = soup.find_all('img')
image_urls = [img['src'] for img in images if 'src' in img.attrs]

# Télécharger les images
for i, img_url in enumerate(image_urls):
    try:
        img_data = requests.get(img_url).content
        img_filename = os.path.join(output_folder, f'image_{i+1}.jpg')
        with open(img_filename, 'wb') as img_file:
            img_file.write(img_data)
        print(f"Downloaded {img_filename}")
    except Exception as e:
        print(f"Could not download {img_url}: {e}")

# Fermer le WebDriver
driver.quit()


Could not download data:image/svg+xml,%3Csvg fill='none' xmlns='http://www.w3.org/2000/svg' viewBox='0 0 16 16'%3E%3Cpath d='M16.0001 7.9996c0 4.418-3.5815 7.9996-7.9995 7.9996S.001 12.4176.001 7.9996 3.5825 0 8.0006 0C12.4186 0 16 3.5815 16 7.9996Z' fill='url(%23paint0_linear_15251_63610)'/%3E%3Cpath d='M16.0001 7.9996c0 4.418-3.5815 7.9996-7.9995 7.9996S.001 12.4176.001 7.9996 3.5825 0 8.0006 0C12.4186 0 16 3.5815 16 7.9996Z' fill='url(%23paint1_radial_15251_63610)'/%3E%3Cpath d='M16.0001 7.9996c0 4.418-3.5815 7.9996-7.9995 7.9996S.001 12.4176.001 7.9996 3.5825 0 8.0006 0C12.4186 0 16 3.5815 16 7.9996Z' fill='url(%23paint2_radial_15251_63610)' fill-opacity='.5'/%3E%3Cpath d='M7.3014 3.8662a.6974.6974 0 0 1 .6974-.6977c.6742 0 1.2207.5465 1.2207 1.2206v1.7464a.101.101 0 0 0 .101.101h1.7953c.992 0 1.7232.9273 1.4917 1.892l-.4572 1.9047a2.301 2.301 0 0 1-2.2374 1.764H6.9185a.5752.5752 0 0 1-.5752-.5752V7.7384c0-.4168.097-.8278.2834-1.2005l.2856-.5712a3.6878 3.6878 0 0 0 .3893-1.6509l-.0

# Scrapping Voursa MARKET

In [6]:
import os
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.voursa.com"
LIST_URL = "https://www.voursa.com/index.cfm?PN={}&gct=1&sct=11&gv=13"
SAVE_DIR = r"C:\Users\Republic Of Computer\Documents\project_ai\scrapp4"

# Votre agent utilisateur
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Brave Chrome/119.0.0.0 Safari/537.36"

# Headers à utiliser pour la requête
headers = {
    "User-Agent": user_agent,
}

# Create the directory if it doesn't exist
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def extract_ad_urls(soup):
    ad_urls = []
    for a_tag in soup.find_all('a', href=True):
        if 'pdtid' in a_tag['href'] and 'adtre' in a_tag['href']:
            ad_urls.append('https://www.voursa.com' + a_tag['href'])
    return ad_urls

def get_image_links(announcement_url):
    response = requests.get(announcement_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    images = []
    
    # Collect images from "photodiv"
    photodiv = soup.find("div", id="photodiv")
    if photodiv:
        for img in photodiv.find_all("img"):
            img_url = img.get("src")
            if img_url and img_url.startswith("/"):
                images.append(BASE_URL + img_url)
            elif img_url:
                images.append(img_url)
    
    return images

def download_image(img_url, save_path):
    response = requests.get(img_url, stream=True, headers=headers)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)

all_announcements = []
all_images = []

# Iterate over the pages from 18 to 26
for page in range(18, 27):
    page_url = LIST_URL.format(page)
    response = requests.get(page_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    announcement_links = extract_ad_urls(soup)
    all_announcements.extend(announcement_links)
    
    for ann_link in announcement_links:
        print(f"Processing announcement: {ann_link} on page: {page}")
        images = get_image_links(ann_link)
        for img_url in images:
            img_name = os.path.basename(img_url)
            save_path = os.path.join(SAVE_DIR, img_name)
            download_image(img_url, save_path)
            print(f"Downloaded {img_url} to {save_path}")


Processing announcement: https://www.voursa.com/annonces.cfm?pdtid=134987&adtre=Corolla 2008 5vitess gazoil mavih chgare on page: 18
Downloaded https://www.voursa.com/includes/toyota/updates/PENTURE/p1.jpg to C:\Users\Republic Of Computer\Documents\project_ai\scrapp4\p1.jpg
Downloaded https://www.voursa.com/includes/toyota/updates/PENTURE/pait_up.jpg to C:\Users\Republic Of Computer\Documents\project_ai\scrapp4\pait_up.jpg
Downloaded https://www.voursa.com/includes/toyota/updates/PENTURE/p3.jpg to C:\Users\Republic Of Computer\Documents\project_ai\scrapp4\p3.jpg
Downloaded https://www.voursa.com/includes/toyota/updates/peinturem/peintrm1.jpg to C:\Users\Republic Of Computer\Documents\project_ai\scrapp4\peintrm1.jpg
Downloaded https://www.voursa.com/includes/toyota/updates/peinturem/peintrm2.jpg to C:\Users\Republic Of Computer\Documents\project_ai\scrapp4\peintrm2.jpg
Downloaded https://www.voursa.com/includes/toyota/updates/peinturem/peintrm3.jpg to C:\Users\Republic Of Computer\Docum

# Previous pages from VOURSA

In [7]:
import os
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.voursa.com"
LIST_URL = "https://www.voursa.com/index.cfm?PN={}&gct=1&sct=11&gv=13"
SAVE_DIR = r"C:\Users\Republic Of Computer\Documents\project_ai\scrapp4"

# Votre agent utilisateur
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Brave Chrome/119.0.0.0 Safari/537.36"

# Headers à utiliser pour la requête
headers = {
    "User-Agent": user_agent,
}

# Create the directory if it doesn't exist
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

def extract_ad_urls(soup):
    ad_urls = []
    for a_tag in soup.find_all('a', href=True):
        if 'pdtid' in a_tag['href'] and 'adtre' in a_tag['href']:
            ad_urls.append('https://www.voursa.com' + a_tag['href'])
    return ad_urls

def get_image_links(announcement_url):
    response = requests.get(announcement_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    images = []
    
    # Collect images from "photodiv"
    photodiv = soup.find("div", id="photodiv")
    if photodiv:
        for img in photodiv.find_all("img"):
            img_url = img.get("src")
            if img_url and img_url.startswith("/"):
                images.append(BASE_URL + img_url)
            elif img_url:
                images.append(img_url)
    
    return images

def download_image(img_url, save_path):
    response = requests.get(img_url, stream=True, headers=headers)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)

all_announcements = []
all_images = []

# Iterate over the pages from 2 to 4
for page in range(2, 5):
    page_url = LIST_URL.format(page)
    response = requests.get(page_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    announcement_links = extract_ad_urls(soup)
    all_announcements.extend(announcement_links)
    
    for ann_link in announcement_links:
        print(f"Processing announcement: {ann_link} on page: {page}")
        images = get_image_links(ann_link)
        for img_url in images:
            img_name = os.path.basename(img_url)
            save_path = os.path.join(SAVE_DIR, img_name)
            download_image(img_url, save_path)
            print(f"Downloaded {img_url} to {save_path}")


Processing announcement: https://www.voursa.com/annonces.cfm?pdtid=335481&adtre=Toyota v8 140000 km on page: 2
Downloaded https://www.voursa.com/includes/toyota/updates/PENTURE/p1.jpg to C:\Users\Republic Of Computer\Documents\project_ai\scrapp4\p1.jpg
Downloaded https://www.voursa.com/includes/toyota/updates/PENTURE/pait_up.jpg to C:\Users\Republic Of Computer\Documents\project_ai\scrapp4\pait_up.jpg
Downloaded https://www.voursa.com/includes/toyota/updates/PENTURE/p3.jpg to C:\Users\Republic Of Computer\Documents\project_ai\scrapp4\p3.jpg
Downloaded https://www.voursa.com/includes/toyota/updates/peinturem/peintrm1.jpg to C:\Users\Republic Of Computer\Documents\project_ai\scrapp4\peintrm1.jpg
Downloaded https://www.voursa.com/includes/toyota/updates/peinturem/peintrm2.jpg to C:\Users\Republic Of Computer\Documents\project_ai\scrapp4\peintrm2.jpg
Downloaded https://www.voursa.com/includes/toyota/updates/peinturem/peintrm3.jpg to C:\Users\Republic Of Computer\Documents\project_ai\scrapp